In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from pprint import pprint
import streamlit as st
from PyPDF2 import PdfReader
import streamlit as st
from loguru import logger
import pandas as pd
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredPowerPointLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.memory import StreamlitChatMessageHistory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
import tiktoken
from loguru import logger
import time
import concurrent.futures
import pandas as pd
import json
import os
from transformers import AutoTokenizer
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredPowerPointLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.memory import StreamlitChatMessageHistory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ragas
import streamlit as st
import PyPDF2
import docx  # For reading DOCX files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from string import Template
import re
import os
import time
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS

# PDF 파일 읽기 함수
def read_pdf(file_path):
    pdf_text = []
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            pdf_text.append(page.extract_text())
    return pdf_text

# DOCX 파일 읽기 함수
def read_docx(file_path):
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

pdf_file_path = "./02-우리_몸이_원하는_삼삼한밥상_Ⅸ_본문(화면용_펼침).pdf"
# docx_file_path = "./data/recipes.docx"
# pdf_file_path2 = "./1.pdf"

# PDF 및 DOCX 파일에서 텍스트 불러오기
pdf_content = read_pdf(pdf_file_path)
# docx_content = read_docx(docx_file_path)
# pdf_content2 = read_pdf(pdf_file_path2)


# PDF 및 DOCX 데이터를 문서 리스트로 변환
documents = []

# pdf_content는 리스트 형태일 가능성이 높으므로 개별 텍스트로 변환
for text in pdf_content:
    if isinstance(text, str):
        documents.append(Document(page_content=text))
    else:
        st.error("PDF 내용이 올바른 문자열 형식이 아닙니다.")

# # docx_content는 하나의 문자열이므로 바로 추가
# if isinstance(docx_content, str):
#     documents.append(Document(page_content=docx_content))
# else:
#     st.error("DOCX 내용이 올바른 문자열 형식이 아닙니다.")

# # pdf_content2도 리스트일 가능성이 있으므로 개별 텍스트로 변환
# for text in pdf_content2:
#     if isinstance(text, str):
#         documents.append(Document(page_content=text))
#     else:
#         st.error("두 번째 PDF 내용이 올바른 문자열 형식이 아닙니다.")


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
import streamlit as st
import pandas as pd
# PDF 및 DOCX 데이터를 문서 리스트로 변환
documents = []

# pdf_content는 리스트 형태일 가능성이 높으므로 개별 텍스트로 변환

# docx_content는 하나의 문자열이므로 바로 추가
if isinstance(docx_content, str):
    documents.append(Document(page_content=docx_content))
else:
    st.error("DOCX 내용이 올바른 문자열 형식이 아닙니다.")

In [24]:
# 텍스트를 청크로 분리하는 함수
def get_text_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=120,  # 청크 크기를 더 줄임
        chunk_overlap=10,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

# 벡터 스토어 생성 함수
def get_vectorstore(_text_chunks):
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sroberta-multitask",
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
    )
    vectordb = FAISS.from_documents(_text_chunks, embeddings)
    return vectordb
# 텍스트 청크 생성
text_chunks = get_text_chunks(documents)

# 벡터 스토어 생성
vectorstore = get_vectorstore(text_chunks)

system_message = """
data = {
    "항목": [
        "이름", "만나이", "성별", "키", "체중", "허리둘레", 
        "이상지질혈증 여부", "당뇨병 여부", 
        "술을 마십니까?", "술을 얼마나 자주 마십니까?", "한 번에 술을 얼마나 마십니까?", 
        "한 번의 술자리에서 7잔 이상을 마시는 횟수", "술을 끊거나 줄이라는 권고를 받은 적이 있습니까?", 
        "최근 1년 동안 음주 문제로 상담을 받아본 적이 있습니까?",
        "고강도 운동 여부", "1주일에 며칠 하십니까?", "한 번 할 때 몇 시간 하십니까?", 
        "중강도 운동 여부", "1주일에 며칠 하십니까?", "한 번 할 때 몇 시간 하십니까?", 
        "걷기나 자전거를 이용하십니까?", "1주일에 며칠 하십니까?", "대략 몇 시간 움직이십니까?",
        "음주 점수", "신체활동 점수", "고혈압 확률",
        "음주 점수 (동나이대/성별 평균)", "신체 활동 점수 (동나이대/성별 평균)", "고혈압 확률 (동나이대/성별 평균)"
    ],
    "값": [
        "김첨지", 38, "남자", "173.0 cm", "89.0 kg", "88.9 cm",
        "없음", "없음",
        "마신다", "월1회정도", "1-2잔",
        "월 1회 미만", "없음", "없음",
        "안 한다", "None", "None", 
        "안 한다", "None", "None",
        "한다", "5일", "2시간 0분",
        "9점", "6.39859점", "21.41%",
        "25.91점", "5.56점", "25.32%"
    ]
}
너는 이 데이터를 읽고 데이터에 알맞은 식단이나 레시피를 소개하주는 영양전문가야.

"""

# 대화형 체인 생성 함수
def get_conversation_chain(_vectorstore):
    llm = ChatOpenAI(
        base_url="http://localhost:1234/v1",
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.3,
        streaming=True
    )

    retriever = _vectorstore.as_retriever(
        search_type='similarity',
        search_kwargs={"k": 5},
        verbose=False
    )

    # 시스템 메시지를 포함하는 프롬프트 템플릿 생성
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "{question}")
    ])

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        chain_type="stuff",
        retriever=retriever, 
        return_source_documents=True,

        verbose=False
    )
    return conversation_chain

# 대화형 체인 생성
conversation_chain = get_conversation_chain(vectorstore)

if 'conversation' not in st.session_state:
    st.session_state.conversation = conversation_chain

def get_prompt_template(query):
    # "고혈압"이 포함된 경우
    if "고혈압" in query:
        prompt = Template(f"""
        사용자가 원하는 기간이 있으면 그 기간, 없으면 하루
        나트륨이 1000mg 이하인 식단을 추천해 주세요.
        무조건 {pdf_file_path} 또는 자료에서만 찾아줘.
        """)
    # "식단"이 포함된 경우
    elif "고혈압" not in query and "식단" in query:
        prompt = Template(f"""
        사용자가 원하는 기간이 있으면 그 기간, 없으면 하루
        나트륨 제한 없이 식단을 추천해 주세요.
        무조건 {pdf_file_path} 또는  자료에서만 찾아줘.
        """)
    # "레시피"가 포함된 경우
    elif "레시피" in query:
        prompt = Template(f"""
        무조건 {pdf_file_path} 또는  자료에서 {query}의 레시피를 찾아서 자세히 설명해 줘.
        """)
    else:
        prompt = Template(f"""
        {pdf_file_path} 또는  자료뿐만 아니라 추가적인 정보를 포함해서 자유롭게 답변해 줘.
        """)
    return prompt.substitute(query=query)

# Chat logic
if query := st.chat_input("식단 또는 레시피를 물어보세요."):
    # 입력된 query를 템플릿화
    formatted_query = get_prompt_template(query)

    with st.chat_message("user"):
        st.markdown(query)

    with st.chat_message("assistant"):
        chain = st.session_state.conversation

        with st.spinner("Thinking..."):
            # 빈 chat_history를 전달
            result = chain({"question": formatted_query, "chat_history": []})
            response = result['answer']
            source_documents = result['source_documents']

            response_container = st.empty()

[09/12/24 18:17:59] INFO     [SentenceTransformer.py:197] >> Load pretrained             ]8;id=145595;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\sentence_transformers\SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=469571;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\sentence_transformers\SentenceTransformer.py#197\197]8;;\
                             SentenceTransformer: jhgan/ko-sroberta-multitask                                      

C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[09/12/24 18:18:02] INFO     [loader.py:74] >> Loading faiss with AVX512 support.                      ]8;id=57204;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py\loader.py]8;;\:]8;id=175755;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py#74\74]8;;\

                    INFO     [loader.py:79] >> Could not load library with AVX512 support due to:      ]8;id=383618;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py\loader.py]8;;\:]8;id=364265;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py#79\79]8;;\
                             ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")                       

                    INFO     [loader.py:86] >> Loading faiss with AVX2 support.                        ]8;id=420616;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py\loader.py]8;;\:]8;id=91847;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py#86\86]8;;\

                    INFO     [loader.py:88] >> Successfully loaded faiss with AVX2 support.            ]8;id=124041;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py\loader.py]8;;\:]8;id=317959;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\faiss\loader.py#88\88]8;;\

2024-09-12 18:18:03.226 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 18:18:03.227 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-09-12 18:18:03.227 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 18:18:03.228 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 18:18:03.229 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 18:18:03.230 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [28]:
from autorag.data.corpus import langchain_documents_to_parquet
corpus_df = langchain_documents_to_parquet(text_chunks, './qnaset7.parquet')

In [32]:
corpus_df.head()
len(corpus_df)

460

In [46]:
import asyncio
import nest_asyncio
from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from llama_index.llms.ollama import Ollama

# nest_asyncio를 사용하여 중첩된 이벤트 루프 허용
nest_asyncio.apply()


In [48]:


import pandas as pd

from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from llama_index.llms.ollama import Ollama


prompt = """
Generate question and answer pairs for the given passage.

Passage:
{{text}}

Number of questions to generate: {{num_questions}}

Example:
[Q]: 곤약 백김치 말이의 레시피는?
[A]: 곤약 백김치 말이의 레시피는 다음과 같습니다:

오징어를 깨끗이 손질하고 물기를 제거합니다.
곤약을 삶아 레몬즙을 넣어 특유의 냄새를 없앱니다.
곤약을 어간장으로 양념하여 맛을 더합니다.
그릇에 곤약, 백김치, 파프리카, 부추, 통후추, 월계수잎을 담고 국물을 부어줍니다.
채소와 달걀을 곁들여 완성합니다..

Result:
"""

corpus_df = pd.read_parquet('./qnaset7.parquet')

llm = Ollama(model='llama3.1:latest', temperature=0.3, timeout=500)
qa_df = make_single_content_qa(corpus_df, content_size=460, qa_creation_func=generate_qa_llama_index,
                            llm=llm, prompt=prompt, question_num_per_content=1)

qa_df.head()

0it [00:00, ?it/s]

[09/12/24 18:21:24] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=100147;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=545296;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             http://localhost:11434/api/chat "HTTP/1.1 200 OK"                                     

[09/12/24 18:21:28] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=104617;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=738619;file://C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             http://localhost:11434/api/chat "HTTP/1.1 200 OK"                                     

0it [00:30, ?it/s]


ReadTimeout: 

In [7]:
qa_df.head()

NameError: name 'qa_df' is not defined

In [9]:
qa_data = pd.read_excel('qnaset1.xlsx')
qa_data.head()

,question,ground_truth
0,새우 두부 계란찜의 주요 재료는 무엇인가요?,"새우두부계란찜의 주요 재료는 연두부, 칵테일새우, 달걀, 생크림, 설탕, 무염버터..."
1,부추 콩가루 찜의 조리 방법은 무엇인가요?,"부추는 깨끗이 씻어 5cm 길이로 썰고 날콩가루를 넣고 섞은 후, 찜기에 면보를 ..."
2,방울토마토 소박이의 양념장 재료는 무엇인가요?,"방울토마토 소박이의 양념장 재료는 고춧가루, 멸치액젓, 다진 마늘, 매실액, 설탕..."
3,고추 된장무침의 재료는 무엇인가요?,"고추 된장무침의 재료는 풋고추, 된장, 다진 마늘, 참기름, 통깨입니다."
4,두부구이의 조리 방법은 무엇인가요?,두부구이는 두부를 도톰하게 썰어 물기를 제거한 후 팬에 기름을 두르고 구운 후 양...


In [11]:
qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      98 non-null     object
 1   ground_truth  98 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [13]:
response = conversation_chain.invoke("새우 두부 계란찜의 주요 재료는 무엇인가요?")
print(response)

NameError: name 'conversation_chain' is not defined

In [15]:
import PyPDF2
import docx  # For reading DOCX files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from string import Template
import re
import os
import time
from langchain.prompts import ChatPromptTemplate
import pandas as pd

# PDF 파일 읽기 함수
def read_pdf(file_path):
    pdf_text = []
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            pdf_text.append(page.extract_text())
    return pdf_text

# DOCX 파일 읽기 함수
def read_docx(file_path):
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# 파일 경로
pdf_file_path = "./02-우리_몸이_원하는_삼삼한밥상_Ⅸ_본문(화면용_펼침).pdf"
docx_file_path = "./data/recipes.docx"
pdf_file_path2 = "./1.pdf"

# PDF 및 DOCX 파일에서 텍스트 불러오기
pdf_content = read_pdf(pdf_file_path)
docx_content = read_docx(docx_file_path)
pdf_content2 = read_pdf(pdf_file_path2)

# 데이터를 딕셔너리 형태로 정리
data = {
    "항목": [
        "이름", "만나이", "성별", "키", "체중", "허리둘레", 
        "이상지질혈증 여부", "당뇨병 여부", 
        "음주 여부", "음주 빈도", "1회 주량", 
        "폭음 횟수", "절주 권고", 
        "음주 상담 유무",
        "고강도 운동 여부", "1주일", "1회 운동 시간", 
        "중강도 운동 여부", "1주일", "1회 운동 시간?", 
        "걷기, 자전거 운동", "1주일", "총 운동 시간",
        "음주 점수", "신체활동 점수", "고혈압 확률",
    ],
    "값": [
        "김첨지", 38, "남자", "173.0 cm", "89.0 kg", "88.9 cm",
        "없음", "없음",
        "마신다", "월1회정도", "1-2잔",
        "월 1회 미만", "없음", "없음",
        "안 한다", "None", "None", 
        "안 한다", "None", "None",
        "한다", "5일", "2시간 0분",
        "9점", "6.39859점", "21.41%",
    ]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# PDF 및 DOCX 데이터를 문서 리스트로 변환
documents = []

# pdf_content는 리스트 형태일 가능성이 높으므로 개별 텍스트로 변환
for text in pdf_content:
    if isinstance(text, str):
        documents.append(Document(page_content=text))

# docx_content는 하나의 문자열이므로 바로 추가
if isinstance(docx_content, str):
    documents.append(Document(page_content=docx_content))

# pdf_content2도 리스트일 가능성이 있으므로 개별 텍스트로 변환
for text in pdf_content2:
    if isinstance(text, str):
        documents.append(Document(page_content=text))

# 텍스트를 청크로 분리하는 함수
def get_text_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,  # 청크 크기를 더 줄임
        chunk_overlap=50,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

# 벡터 스토어 생성 함수
def get_vectorstore(_text_chunks):
    embeddings = HuggingFaceEmbeddings(
        model_name="jhgan/ko-sroberta-multitask",
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
    )
    vectordb = FAISS.from_documents(_text_chunks, embeddings)
    return vectordb

# 시스템 프롬프트 템플릿 생성
system_message = """
data = {
    "항목": [
        "이름", "만나이", "성별", "키", "체중", "허리둘레", 
        "이상지질혈증 여부", "당뇨병 여부", 
        "술을 마십니까?", "술을 얼마나 자주 마십니까?", "한 번에 술을 얼마나 마십니까?", 
        "한 번의 술자리에서 7잔 이상을 마시는 횟수", "술을 끊거나 줄이라는 권고를 받은 적이 있습니까?", 
        "최근 1년 동안 음주 문제로 상담을 받아본 적이 있습니까?",
        "고강도 운동 여부", "1주일에 며칠 하십니까?", "한 번 할 때 몇 시간 하십니까?", 
        "중강도 운동 여부", "1주일에 며칠 하십니까?", "한 번 할 때 몇 시간 하십니까?", 
        "걷기나 자전거를 이용하십니까?", "1주일에 며칠 하십니까?", "대략 몇 시간 움직이십니까?",
        "음주 점수", "신체활동 점수", "고혈압 확률"
    ],
    "값": [
        "김첨지", 38, "남자", "173.0 cm", "89.0 kg", "88.9 cm",
        "없음", "없음",
        "마신다", "월1회정도", "1-2잔",
        "월 1회 미만", "없음", "없음",
        "안 한다", "None", "None", 
        "안 한다", "None", "None",
        "한다", "5일", "2시간 0분",
        "9점", "6.39859점", "21.41%",
    ]
}
너는 이 데이터를 읽고 데이터에 알맞은 식단이나 레시피를 소개하주는 영양전문가야.
"""

# 텍스트 청크 생성
text_chunks = get_text_chunks(documents)

# 벡터 스토어 생성
vectorstore = get_vectorstore(text_chunks)

# 대화형 체인 생성 함수
def get_conversation_chain(_vectorstore):
    llm = ChatOpenAI(
        base_url="http://localhost:1234/v1",
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.3,
        streaming=True
    )

    retriever = _vectorstore.as_retriever(
        search_type='similarity',
        search_kwargs={"k": 5},
        verbose=False
    )

    # 시스템 메시지를 포함하는 프롬프트 템플릿 생성
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "{question}")
    ])

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        chain_type="stuff",
        retriever=retriever, 
        return_source_documents=True,
        verbose=False
    )
    return conversation_chain

# 대화형 체인 생성
conversation_chain = get_conversation_chain(vectorstore)

# 사용자 질의 처리 함수
def process_user_query(query):
    formatted_query = get_prompt_template(query)
    result = conversation_chain({"question": formatted_query, "chat_history": []})
    response = result['answer']
    return response


C:\Users\CAWCAW\AppData\Local\Temp\ipykernel_15720\241701327.py:101: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
C:\Users\CAWCAW\anaconda3\envs\dl\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
response = conversation_chain.invoke({
    "question": "새우 두부 계란찜의 주요 재료는 무엇인가요?", 
    "chat_history": []
})
print(response)

{'question': '새우 두부 계란찜의 주요 재료는 무엇인가요?', 'chat_history': [], 'answer': '새우 두부 계란찜의 주요 재료는 연두부, 칵테일 새우, 달걀, 생크림, 설탕, 무염버터입니다.', 'source_documents': [Document(page_content='새우 두부 계란찜\n기본 정보\n일련번호: 28\n조리방법: 찌기\n요리종류: 반찬\n영양 정보\n중량(1인분): g\n열량: 220 kcal\n탄수화물: 3g\n단백질: 14g\n지방: 17g\n나트륨: 99mg\n재료 정보\n새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)\n고명\n시금치 10g(3줄기)\n조리 방법\n단계 1: 1. 손질된 새우를 끓는 물에 데쳐 건진다.a\n이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_1.png\n단계 2: 2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)를 함께 섞어 그릇에 담는다.b\n이미지: http://www.foodsafetykorea.go.kr/uploadimg/cook/20_00028_2.png'), Document(page_content='새우살을 채운 두부 스프(미네스트로네)\n기본 정보\n일련번호: 1040\n조리방법: 끓이기\n요리종류: 일품\n영양 정보\n중량(1인분): g\n열량: 117 kcal\n탄수화물: 5.9g\n단백질: 13.8g\n지방: 4.6g\n나트륨: 361mg\n재료 정보\n새우 35g, 두부 50g, 당근 10g, 돼지호박 10g, 참기름 1g, 생강 1g, 양파 10g, 마늘다진것 1g, 닭육수 70g, 굴소스 3g, 간장 2g, 소금적당량[새우 마리네이션]달걀흰자 5g, 전분 2g, 소금 적당량\n조리 방법\n단계 1: 1. 새우살을 다져서 양념한다.\n이미

In [46]:
qa_data['ground_truth'] = qa_data['answer']

# 'answer'와 'contexts' 열을 conversation_chain과 retriever로 채움
qa_data['answer'] = qa_data['question'].apply(lambda x: conversation_chain.invoke({
    "question": x, 
    "chat_history": []
}))  # answer 부분만 추출

# 'contexts' 열에 관련 문서의 내용을 추가
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)])

# 데이터 확인
qa_data.head()

NameError: name 'retriever' is not defined

In [48]:
qa_data.head()

,question,ground_truth,answer,contexts
0,새우 두부 계란찜의 주요 재료는 무엇인가요?,"새우두부계란찜의 주요 재료는 연두부 75g(3/4모), 칵테일새우 20g(5마리)...","{'question': ' 새우 두부 계란찜의 주요 재료는 무엇인가요?', 'cha...",[]
1,부추 콩가루 찜의 조리 방법은 무엇인가요?,부추 콩가루 찜의 조리 방법은 다음과 같습니다:\n\n1. 부추는 깨끗이 씻어 물기...,"{'question': ' 부추 콩가루 찜의 조리 방법은 무엇인가요?', 'chat...",[]
2,방울토마토 소박이의 양념장 재료는 무엇인가요?,"고춧가루, 멸치액젓, 다진 마늘, 매실액, 설탕, 물, 통깨입니다.","{'question': ' 방울토마토 소박이의 양념장 재료는 무엇인가요?', 'ch...",[]
3,고추 된장무침의 재료는 무엇인가요?,"고추 된장무침의 재료는 다음과 같습니다: 오이고추(40g), 영양부추(5g), 연근...","{'question': ' 고추 된장무침의 재료는 무엇인가요?', 'chat_his...",[]
4,두부구이의 조리 방법은 무엇인가요?,두부를 1.5cm 두께로 잘라 물기를 뺀다.\n프라이팬에 올리브유를 두르고 두부를 ...,"{'question': ' 두부구이의 조리 방법은 무엇인가요?', 'chat_his...",[]


In [52]:
# answer 열에 있는 값을 문자열로 변환
def extract_text_from_document(value):
    if isinstance(value, Document):
        return value.page_content
    elif isinstance(value, str):
        return value
    else:
        return str(value)  # 다른 유형은 문자열로 변환

# apply 함수를 사용하여 answer 열의 모든 값을 문자열로 변환
qa_data['answer'] = qa_data['answer'].apply(extract_text_from_document)

# Pandas DataFrame을 Dataset으로 변환
from datasets import Dataset

dataset = Dataset.from_pandas(qa_data)

# 결과 확인
print(dataset[1])


{'question': ' 부추 콩가루 찜의 조리 방법은 무엇인가요?', 'ground_truth': '부추 콩가루 찜의 조리 방법은 다음과 같습니다:\n\n1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣고 고루 섞이도록 버무린다.\n2. 찜기에 면보를 깔고 부추를 넣은 후 김이 오르게 쪄서 파랗게 익힌다.\n3. 저염간장에 다진 대파, 다진 마늘, 고춧가루, 요리당 , 참기름, 참깨를 섞어 양념장을 만들고 찐 부추는 그릇에 담아낸다.', 'answer': "{'question': ' 부추 콩가루 찜의 조리 방법은 무엇인가요?', 'chat_history': [], 'answer': '부추 콩가루 찜의 조리 방법은 다음과 같습니다:\\n\\n1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣고 고루 섞이도록 버무립니다.\\n2. 찜기에 면보를 깔고 부추를 넣은 후 김이 오르게 쪄서 파랗게 익힙니다.\\n3. 저염간장에 다진 대파, 다진 마늘, 고춧가루, 요리당, 참기름, 참깨를 섞어 양념장을 만들고 찐 부추는 그릇에 담아냅니다.', 'source_documents': [Document(page_content='부추 콩가루 찜\\n기본 정보\\n일련번호: 29\\n조리방법: 찌기\\n요리종류: 반찬\\n영양 정보\\n중량(1인분): g\\n열량: 215 kcal\\n탄수화물: 20g\\n단백질: 14g\\n지방: 9g\\n나트륨: 240mg\\n재료 정보\\n[1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\\n·양념장 : 저염간장 3g(2/3작은술), 다진 대파 5g(1작은술), 다진 마늘 2g(1/2쪽), 고춧가루 2g(1/3작은술), 요리당 2g(1/3작은술), 참기름 2g(1/3작은술), 참깨 약간\\n조리 방법\\n단계 1: 1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣고 고루 섞이도록 버무린다.\\n이미지: http://www.foodsafet

In [54]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
from langchain.llms.ollama import Ollama
from ragas import evaluate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

langchain_llm =  ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="gemma2")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result

Evaluating:   0%|          | 0/392 [00:00<?, ?it/s]

Exception raised in Job[82]: TimeoutError()
Exception raised in Job[228]: TimeoutError()
Exception raised in Job[328]: TimeoutError()
Exception raised in Job[346]: TimeoutError()
Exception raised in Job[230]: TimeoutError()
Exception raised in Job[292]: TimeoutError()
Exception raised in Job[62]: TimeoutError()
Exception raised in Job[224]: TimeoutError()
Exception raised in Job[320]: TimeoutError()
Exception raised in Job[286]: TimeoutError()
Exception raised in Job[229]: TimeoutError()
Exception raised in Job[174]: TimeoutError()
Exception raised in Job[65]: TimeoutError()
Exception raised in Job[225]: TimeoutError()
Exception raised in Job[314]: TimeoutError()
Exception raised in Job[289]: TimeoutError()
